In [1]:
import pickle

- xgb : 0.72
- cb : 0.71
- deep : 0.65

In [2]:
reply_to_idx_dict = dict()
with open("../data/test_replies.txt", "r") as f:
    for line in f:
        idx, reply = line.strip().split("\t")
        reply_to_idx_dict[reply] = idx

In [3]:
with open("../data/test_replies.txt", "r") as f:
    test_replies = [line.strip().split("\t")[1] for line in f]

In [4]:
xgb_result = pickle.load(open("../dump/xgb_result.pkl", "rb"))
cb_result = pickle.load(open("../dump/cb_result.pkl", "rb"))
deep_result = pickle.load(open("../dump/deep_result.pkl", "rb"))

In [5]:
with open("../data/test_queries.txt", "r") as f:
    test_queries = [line.strip().split("\t")[1] for line in f]

In [6]:
from tqdm import tqdm

In [7]:
predictions = list()

In [8]:
for query in tqdm(test_queries):
    xgb_dict = {r: score for r, score in xgb_result[query]}
    cb_dict = {r: score for r, score in cb_result[query]}
    deep_dict = {r: score for r, score in deep_result[query]}
    total_dict = dict()
    for reply in test_replies:
        total_dict[reply] = xgb_dict[reply]*0.72 + cb_dict[reply]*0.71 + deep_dict[reply]*0.65
    predictions.append([query, sorted(total_dict.items(), key=lambda x: x[1], reverse=True)[0][0]])

100%|██████████| 600/600 [00:00<00:00, 1613.49it/s]


In [9]:
submission = list()
for p in predictions:
    reply = p[1]
    submission.append(reply)

In [11]:
import pandas as pd

In [12]:
submission_xgb = pd.DataFrame([(query, reply) for query, reply in zip(test_queries, submission)])

In [17]:
import pandas as pd

In [18]:
s = pd.read_csv("../submission/sample_submission.csv")

In [20]:
s["id_script"] = pd.Series(submission)

In [22]:
s.to_csv("../submission/submission_1.csv", index=False)

In [24]:
xgb_predictions = list()
for query in tqdm(test_queries):
    xgb_predictions.append([query, sorted(xgb_result[query], key=lambda x: x[1], reverse=True)[0][0]])

100%|██████████| 600/600 [00:00<00:00, 13754.52it/s]


In [26]:
s = pd.read_csv("../submission/sample_submission.csv")
s["id_script"] = pd.Series([reply_to_idx_dict[r] for r in [p[1] for p in xgb_predictions]])

In [28]:
s.to_csv("../submission/submission_xgb.csv", index=False)